In [ ]:
!pip install biopython

In [ ]:
!pip install --upgrade --force-reinstall transformers

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from collections import defaultdict
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import seaborn as sns
import matplotlib.pyplot as plt
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import get_scheduler
from transformers import AutoModel, AutoConfig,AutoModelForSequenceClassification 
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup, get_constant_schedule_with_warmup
import gc
import torch
from tqdm.auto import tqdm
import tokenizers
import time
import math
import string
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")
import scipy as sp
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset
from transformers import BertModel, BertTokenizer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, log_loss, precision_score, recall_score, confusion_matrix, matthews_corrcoef, classification_report
from sklearn.model_selection import StratifiedKFold
import wandb
# CPMP: declare the two GPUs
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"
# CPMP: avoids some issues when using more than one worker
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import matplotlib.pyplot as plt
import Bio
from Bio import SeqIO
import os
from transformers import EsmModel
from transformers import EsmTokenizer

In [ ]:
def protbert_prediction(file_path):
    # define the path and output directory
    Output_Directory = f'PROTBERT/'
    if not os.path.exists(Output_Directory):
        os.makedirs(Output_Directory)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # configuration for the protbert model
    class CFG:
        apex=True
        print_freq=20
        num_workers=1 
        model="Rostlab/prot_bert"
        gradient_checkpointing=False
        scheduler='cosine' # ['linear', 'cosine', 'constant']
        batch_scheduler=True
        num_warmup_steps=0
        epochs=6
        num_cycles=1
        encoder_lr=4.09e-5
        decoder_lr= 4.49e-5
        batch_size=32
        dropout = 0
        # Model information for the rostlab/prot_bert model
        total_layers = 30 # found from the explanation below
        initial_layers = 5 
        layers_per_block = 16 
        
        # Parameters of the model can be frozen, total parameters of a model can be found after loading it using:
        #for i,(name, param) in enumerate(list(model.named_parameters())): print(i,name)
        # parameters can then be frozen using:
        #for name, param in list(model.named_parameters())\[:CFG.initial_layers+CFG.layers_per_block*CFG.num_freeze_layers]: param.requires_grad = False
        # Setting number of frozen layers:
        num_freeze_layers = total_layers - 2
        #configuration of the model parameters, the seed, folds for groupkfold splitting, 
        #as well as the dimension reduction used by the second last linear layer for embedding dimension reduction.
        min_lr=1e-6
        eps=1e-08
        betas=(0.9, 0.999)
        max_len=100
        weight_decay=0.00175
        gradient_accumulation_steps=4
        max_grad_norm=1000
        target_cols=['labels']
        seed=42
        n_fold=10
        trn_fold=[0,1,2,3,4,5,6,7,8,9]
        train=True
        pca_dim = 256
    # Tokenizer for protbert
    tokenizer = AutoTokenizer.from_pretrained('PROTBERT/tokenizer/')
    CFG.tokenizer = AutoTokenizer.from_pretrained('PROTBERT/tokenizer/')
    
    # model
    class MeanPooling(nn.Module):
        def __init__(self):
            super(MeanPooling, self).__init__()    
        def forward(self, last_hidden_state, attention_mask):
            #adds a new dimension after the last index of the attention_mask, then expands it to the size of the last hidden state.   
            input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
            sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
            sum_mask = input_mask_expanded.sum(1)
            #clamps all elements in the sum_mask such that the minimum value is 1e-9, making any zeroes a small number instead.
            sum_mask = torch.clamp(sum_mask, min=1e-9)
            #returns the mean of the embeddings
            mean_embeddings = sum_embeddings / sum_mask
            return mean_embeddings
    

    class CustomModel(nn.Module):
        def __init__(self, cfg, config_path=None, pretrained=False):
            super().__init__()
            self.cfg = cfg
            #configuration for the model, loads the model specified in the config section at the top of the notebook.
            if config_path is None:
                self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
                self.config.hidden_dropout = cfg.dropout
                self.config.hidden_dropout_prob = cfg.dropout
                self.config.attention_dropout = cfg.dropout
                self.config.attention_probs_dropout_prob = cfg.dropout
            else:
                self.config = torch.load(config_path)
            if pretrained:
                self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
            else:
                self.model = AutoModel.from_config(self.config)
            
            if self.cfg.gradient_checkpointing:
                self.model.gradient_checkpointing_enable()
            
            #returns the mean of the embeddings, as specified in the MeanPooling() function.
            self.pool = MeanPooling()
            #linear layer, reduces the dimenzionality of the hidden_size (in this case 1024) specified in the config file, 
            #and reduces it to the pca_dim specified in the config file (In this case 64)
            self.fc1 = nn.Linear(self.config.hidden_size, self.cfg.pca_dim)
            #second linear layer, reduces the dimensonality from the pca_dim*6 (6, for 6 features) to 1 (dTm) nn.Linear changed to nn.Softmax for classification
            self.fc2 = nn.Linear(self.cfg.pca_dim, 1)
            #initializes weights of the linear layers
            self._init_weights(self.fc1)
            self._init_weights(self.fc2)
        
        
        def _init_weights(self, module):
            if isinstance(module, nn.Linear):
                module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
                if module.bias is not None:
                    module.bias.data.zero_()
            elif isinstance(module, nn.Embedding):
                module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
                if module.padding_idx is not None:
                    module.weight.data[module.padding_idx].zero_()
            elif isinstance(module, nn.LayerNorm):
                module.bias.data.zero_()
                module.weight.data.fill_(1.0)
        
        def feature(self, batch):
            # CPMP comment: pass inputs explicitly
            outputs = self.model(batch['input_ids'],batch['attention_mask'])
            last_hidden_states = outputs[0]
            feature = self.pool(last_hidden_states, batch['attention_mask'])
            return feature

        def forward(self, batch):
            # CPMP comment: change code to read inputs from a single dictionary
            feature = self.fc1(self.feature(batch))
        
            #feature = torch.cat((feature),axis=-1)
            output = self.fc2(feature)
            return output
        # prepare the sequence input for prediction by the pretrained protbert model
    def prepare_input(cfg, text):
        inputs = tokenizer.encode_plus(
            text, 
            return_tensors=None, 
            add_special_tokens=True, 
            max_length=cfg.max_len,
            pad_to_max_length=True,
            truncation=True)
        for k, v in inputs.items():
            inputs[k] = torch.tensor(v, dtype=torch.long)
        return inputs
    # test dataset
    CFG.path = Output_Directory
    CFG.config_path = CFG.path+'config.pth' 
    class TestDataset(Dataset):
        def __init__(self, cfg, df):
            self.cfg = cfg
            self.text = df['sequence'].values
        def __len__(self):
            return len(self.text)

        def __getitem__(self, item):
            #prepares the text inputs for use in the bert model (tokenize, pad, truncate, encode)
            inputs1 = prepare_input(self.cfg, self.text[item])
            #gets the labels for each item
            #returns a dict of all inputs
            return {'input_ids' : inputs1['input_ids'], 
                    'attention_mask' : inputs1['attention_mask']}
    # inference on file
    def inference_fn(test_loader, model, device):
        preds = []
        model.eval()
        model.to(device)
        # CPMP: using 2 GPU
        #model = nn.DataParallel(model)
        tk0 = tqdm(test_loader, total=len(test_loader))
        # CPMP: iterating on batch dictionaries
        for batch in tk0:
            for k, v in batch.items():
                batch[k] = v.to(device)
            with torch.no_grad():
                y_preds = model(batch)
            preds.append(y_preds.to('cpu').numpy())
        predictions = np.concatenate(preds)
        return predictions
    # add spaces to the FASTA sequence, protein NLP models were trained on space separated sequences (A G A G)
    def add_spaces(x):
        return " ".join(list(x))

    # load the test file
    def load_test(file_path):
        with open(file_path) as fasta_file:
            identifiers = []
            lengths = []
            seq = []
            for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
                seq.append(str(seq_record.seq))
                identifiers.append(seq_record.id)
                lengths.append(len(seq_record.seq))
        test = pd.DataFrame()
        test['sequence'] = seq
        test['len'] = lengths
        test['identifiers'] = identifiers
        test.drop_duplicates(inplace=True)
        test.reset_index(drop=True,inplace=True)
        test['sequence'] = test.sequence.map(add_spaces)
        return test
    # protbert prediction on the fasta file
    def pred(test):
        test_dataset = TestDataset(CFG, test)
        test_loader = DataLoader(test_dataset,
                                 batch_size=CFG.batch_size,
                                 shuffle=False,
                                 num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
    
        predictions_ = []
        for fold in CFG.trn_fold:
            model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
            state = torch.load(CFG.path+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
                               map_location=torch.device('cpu'))
            model.load_state_dict(state['model'])
            prediction = inference_fn(test_loader, model, device)
            predictions_.append(prediction)
            del model, state, prediction; gc.collect()
            torch.cuda.empty_cache()
        predictions = np.mean(predictions_, axis=0)
        test['predictions']  = predictions
        return predictions
    #load the fasta file
    test = load_test(file_path)
    #protbert predict the fasta sequences
    test['predictions'] = pred(test)
    test['protbert'] =torch.sigmoid(torch.tensor(test['predictions']))
    #save a csv for the protbert predictions
    test.to_csv('protbert_prediction')
    return test['protbert']

In [ ]:
#protbert_predictions = protbert('XUAMP_testing_datasets/XU_AMP.fasta')

In [ ]:
def esm_prediction(file_path):
    # define the path and output directory
    Output_Directory = f'ESM/'
    if not os.path.exists(Output_Directory):
        os.makedirs(Output_Directory)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # configuration for the esm model
    class CFG:
        apex=True
        print_freq=20
        num_workers=1 
        model="facebook/esm2_t33_650M_UR50D"
        gradient_checkpointing=False
        scheduler='cosine' # ['linear', 'cosine', 'constant']
        batch_scheduler=True
        num_warmup_steps=0
        epochs=6
        num_cycles=1
        encoder_lr=4.09e-5
        decoder_lr= 4.49e-5
        batch_size=32
        dropout = 0
        total_layers = int(model.split('_')[1][1:])
        initial_layers = 2 
        layers_per_block = 16
        num_freeze_layers = total_layers - 2
        min_lr=1e-6
        eps=1e-08
        betas=(0.9, 0.999)
        max_len=100
        weight_decay=0.00175
        gradient_accumulation_steps=4
        max_grad_norm=1000
        target_cols=['labels']
        seed=42
        n_fold=10
        trn_fold=[0,1,2,3,4,5,6,7,8,9]
        train=True
        pca_dim = 256
    # Tokenizer for esm
    tokenizer = AutoTokenizer.from_pretrained('ESM/tokenizer/')
    CFG.tokenizer = AutoTokenizer.from_pretrained('ESM/tokenizer/')
    
    # model
    class MeanPooling(nn.Module):
        def __init__(self):
            super(MeanPooling, self).__init__()    
        def forward(self, last_hidden_state, attention_mask):
            #adds a new dimension after the last index of the attention_mask, then expands it to the size of the last hidden state.   
            input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
            sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
            sum_mask = input_mask_expanded.sum(1)
            #clamps all elements in the sum_mask such that the minimum value is 1e-9, making any zeroes a small number instead.
            sum_mask = torch.clamp(sum_mask, min=1e-9)
            #returns the mean of the embeddings
            mean_embeddings = sum_embeddings / sum_mask
            return mean_embeddings
    

    class CustomModel(nn.Module):
        def __init__(self, cfg, config_path=None, pretrained=False):
            super().__init__()
            self.cfg = cfg
            #configuration for the model, loads the model specified in the config section at the top of the notebook.
            if config_path is None:
                self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
                self.config.hidden_dropout = cfg.dropout
                self.config.hidden_dropout_prob = cfg.dropout
                self.config.attention_dropout = cfg.dropout
                self.config.attention_probs_dropout_prob = cfg.dropout
            else:
                self.config = torch.load(config_path)
            if pretrained:
                self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
            else:
                self.model = AutoModel.from_config(self.config)
            
            if self.cfg.gradient_checkpointing:
                self.model.gradient_checkpointing_enable()
            
            #returns the mean of the embeddings, as specified in the MeanPooling() function.
            self.pool = MeanPooling()
            #linear layer, reduces the dimenzionality of the hidden_size (in this case 1024) specified in the config file, 
            #and reduces it to the pca_dim specified in the config file (In this case 64)
            self.fc1 = nn.Linear(self.config.hidden_size, self.cfg.pca_dim)
            #second linear layer, reduces the dimensonality from the pca_dim*6 (6, for 6 features) to 1 (dTm) nn.Linear changed to nn.Softmax for classification
            self.fc2 = nn.Linear(self.cfg.pca_dim, 1)
            #initializes weights of the linear layers
            self._init_weights(self.fc1)
            self._init_weights(self.fc2)
        
        
        def _init_weights(self, module):
            if isinstance(module, nn.Linear):
                module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
                if module.bias is not None:
                    module.bias.data.zero_()
            elif isinstance(module, nn.Embedding):
                module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
                if module.padding_idx is not None:
                    module.weight.data[module.padding_idx].zero_()
            elif isinstance(module, nn.LayerNorm):
                module.bias.data.zero_()
                module.weight.data.fill_(1.0)
        
        def feature(self, batch):
            # CPMP comment: pass inputs explicitly
            outputs = self.model(batch['input_ids'],batch['attention_mask'])
            last_hidden_states = outputs[0]
            feature = self.pool(last_hidden_states, batch['attention_mask'])
            return feature

        def forward(self, batch):
            # CPMP comment: change code to read inputs from a single dictionary
            feature = self.fc1(self.feature(batch))
        
            #feature = torch.cat((feature),axis=-1)
            output = self.fc2(feature)
            return output
        # prepare the sequence input for prediction by the pretrained protbert model
    def prepare_input(cfg, text):
        inputs = tokenizer.encode_plus(
            text, 
            return_tensors=None, 
            add_special_tokens=True, 
            max_length=cfg.max_len,
            pad_to_max_length=True,
            truncation=True)
        for k, v in inputs.items():
            inputs[k] = torch.tensor(v, dtype=torch.long)
        return inputs
    # test dataset
    CFG.path = Output_Directory
    CFG.config_path = CFG.path+'config.pth' 
    class TestDataset(Dataset):
        def __init__(self, cfg, df):
            self.cfg = cfg
            self.text = df['sequence'].values
        def __len__(self):
            return len(self.text)

        def __getitem__(self, item):
            #prepares the text inputs for use in the bert model (tokenize, pad, truncate, encode)
            inputs1 = prepare_input(self.cfg, self.text[item])
            #gets the labels for each item
            #returns a dict of all inputs
            return {'input_ids' : inputs1['input_ids'], 
                    'attention_mask' : inputs1['attention_mask']}
    # inference on file
    def inference_fn(test_loader, model, device):
        preds = []
        model.eval()
        model.to(device)
        # CPMP: using 2 GPU
        #model = nn.DataParallel(model)
        tk0 = tqdm(test_loader, total=len(test_loader))
        # CPMP: iterating on batch dictionaries
        for batch in tk0:
            for k, v in batch.items():
                batch[k] = v.to(device)
            with torch.no_grad():
                y_preds = model(batch)
            preds.append(y_preds.to('cpu').numpy())
        predictions = np.concatenate(preds)
        return predictions
    # add spaces to the FASTA sequence, protein NLP models were trained on space separated sequences (A G A G)
    def add_spaces(x):
        return " ".join(list(x))

    # load the test file
    def load_test(file_path):
        with open(file_path) as fasta_file:
            identifiers = []
            lengths = []
            seq = []
            for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
                seq.append(str(seq_record.seq))
                identifiers.append(seq_record.id)
                lengths.append(len(seq_record.seq))
        test = pd.DataFrame()
        test['sequence'] = seq
        test['len'] = lengths
        test['identifiers'] = identifiers
        test.drop_duplicates(inplace=True)
        test.reset_index(drop=True,inplace=True)
        test['sequence'] = test.sequence.map(add_spaces)
        return test
    # protbert prediction on the fasta file
    def pred(test):
        test_dataset = TestDataset(CFG, test)
        test_loader = DataLoader(test_dataset,
                                 batch_size=CFG.batch_size,
                                 shuffle=False,
                                 num_workers=CFG.num_workers, pin_memory=True, drop_last=False)  
        predictions_ = []
        for fold in CFG.trn_fold:
            model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
            state = torch.load(CFG.path+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
                               map_location=torch.device('cpu'))
            model.load_state_dict(state['model'])
            prediction = inference_fn(test_loader, model, device)
            predictions_.append(prediction)
            del model, state, prediction; gc.collect()
            torch.cuda.empty_cache()
        predictions = np.mean(predictions_, axis=0)
        test['predictions']  = predictions
        return predictions
    #load the fasta file
    test = load_test(file_path)
    #protbert predict the fasta sequences
    test['predictions'] = pred(test)
    test['esm'] =torch.sigmoid(torch.tensor(test['predictions']))
    #save a csv for the protbert predictions
    test.to_csv('esm_prediction')
    return test['esm']

In [ ]:
def amp_predictor(